In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn as sk

training = pd.read_csv('setEntrenamiento.csv', low_memory = False)
test = pd.read_csv('setTesteo.csv', low_memory = False)
kaggle = pd.read_csv('setKaggle.csv', low_memory = False)

In [3]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, r2_score, classification_report, mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import roc_auc_score, roc_curve

In [4]:
training["person"] = training["person"].astype(str)
test["person"] = test["person"].astype(str)
kaggle["person"] = kaggle["person"].astype(str)

In [16]:
kaggle["person"]

0        00091926
1        00091a7a
2        000ba417
3        000e4d9e
4        000e619d
5        001001be
6        0010e89a
7        0016c4b5
8        001804a2
9        001a2273
10       00202cdb
11       0022965d
12       0027574e
13       00295d74
14       002aea56
15       002ed810
16       002fb981
17       00344873
18       0038a117
19       003a05bf
20       003d07ef
21       003e7507
22       00408349
23       0041fec6
24       0043a48e
25       00546f4c
26       00556ee5
27       0059a996
28       005af223
29       005e06ef
           ...   
19385    ff9e0500
19386    ff9ebf5a
19387    ff9fc164
19388    ffa0b2f3
19389    ffa0d7a0
19390    ffa28549
19391    ffacfff0
19392    ffaee083
19393    ffb4618c
19394    ffb46215
19395    ffb7c1a6
19396    ffba4aec
19397    ffbb282f
19398    ffbb90a9
19399    ffbfc511
19400    ffc0f2a0
19401    ffccd5a1
19402    ffcf952b
19403    ffd41214
19404    ffd62616
19405    ffda14ca
19406    ffdafc72
19407    ffddd0ec
19408    ffe53446
19409    f

In [6]:
label_array = np.array(training['label'])
data_array = np.array(training.drop(columns=['label', 'person']))
kaggle_array = np.array(kaggle.drop(columns='person'))

In [15]:
kaggle_array

array([[372.,   0.,   2., ...,   0.,   0.,   0.],
       [  3.,   0.,   0., ...,   0.,   0.,   0.],
       [153.,   1.,   6., ...,  22.,   1.,   0.],
       ...,
       [  0.,   1.,   0., ...,   0.,   0.,   0.],
       [ 42.,  12.,   1., ...,   2.,   0.,   0.],
       [  4.,   1.,   1., ...,   0.,   0.,   0.]])

In [40]:
#CrossValidation y Grid Search RandomForest
x_train, x_test, y_train, y_test = train_test_split(data_array, label_array, test_size = 0, random_state = 0)
classifier = RandomForestClassifier(n_estimators = 300, random_state= 0)
all_accuracies = cross_val_score(estimator = classifier, X= x_train, y = y_train, cv = 5)
print(all_accuracies)

[0.94877156 0.94797386 0.94901961 0.94875817 0.94849673]


In [41]:
print(all_accuracies.mean())

0.9486039858006279


In [42]:
print(all_accuracies.std())

0.00035585480691379323


In [70]:
#Empieza el gridSearch
grid_param = {
    "n_estimators": [300, 500, 800],
    "criterion": ["gini", "entropy"],
    "bootstrap": [True, False]
}

In [47]:
from  sklearn.metrics import SCORERS

In [48]:
#metricas que se pueden usar
SCORERS.keys()

['precision_samples',
 'f1_weighted',
 'balanced_accuracy',
 'f1_samples',
 'f1',
 'adjusted_mutual_info_score',
 'precision',
 'normalized_mutual_info_score',
 'neg_mean_squared_error',
 'precision_micro',
 'neg_mean_squared_log_error',
 'recall_samples',
 'homogeneity_score',
 'precision_weighted',
 'fowlkes_mallows_score',
 'average_precision',
 'roc_auc',
 'adjusted_rand_score',
 'recall_macro',
 'v_measure_score',
 'completeness_score',
 'r2',
 'recall_weighted',
 'recall',
 'precision_macro',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'mutual_info_score',
 'f1_macro',
 'f1_micro',
 'brier_score_loss',
 'recall_micro',
 'neg_median_absolute_error',
 'explained_variance',
 'accuracy']

In [50]:
gd_sr = GridSearchCV(estimator=classifier, param_grid=grid_param, scoring="roc_auc", cv=5, n_jobs=1)

In [51]:
#Esto va a tardar
gd_sr.fit(x_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=1,
       param_grid={'n_estimators': [300, 500, 800], 'bootstrap': [True, False], 'criterion': ['gini', 'entropy']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [54]:
#muestra el mejor hiperparametro
best_parameters = gd_sr.best_params_
print(best_parameters)

{'n_estimators': 800, 'bootstrap': True, 'criterion': 'entropy'}


In [72]:
gd_sr.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=800, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [71]:
#Muestra el mejor score
gd_sr.best_score_

0.8173777837905141

In [7]:
x_train, x_test, y_train, y_test = train_test_split(data_array, label_array, test_size = 0.33, random_state = 7)

In [9]:
def logisticReg(x_train, x_test, y_train, y_test):
    logReg = LogisticRegression(solver='sag', random_state=1)
    logReg.fit(x_train, y_train)
    y_pred = logReg.predict_proba(x_test)
    #return np.sqrt(mean_squared_error(y_test, y_pred))
    return roc_auc_score(y_test, y_pred[:,1])
    #return y_pred
#TAMBIEN PUEDEN USAR PARA VER EL ACCURACY
#ACCURACY = accuracy_score(y_test, y_pred)

In [17]:
def knn(k, x_train, x_test, y_train, y_test):
    knn = KNeighborsClassifier(n_neighbors=k, metric='manhattan')
    knn.fit(x_train, y_train)
    y_pred = knn.predict_proba(x_test)
    #return np.sqrt(mean_squared_error(y_test, y_pred))
    return roc_auc_score(y_test, y_pred[:,1])
#TAMBIEN PUEDEN USAR PARA VER EL ACCURACY
#ACCURACY = accuracy_score(y_test, y_pred)

In [19]:
def decisionTree(max_depth, x_train, x_test, y_train, y_test):
    dt = DecisionTreeClassifier(max_depth=max_depth)
    dt.fit(x_train, y_train)
    y_pred = dt.predict_proba(x_test)
#     return np.sqrt(mean_squared_error(y_test, y_pred))
    return roc_auc_score(y_test, y_pred[:,1])
#TAMBIEN PUEDEN USAR PARA VER EL ACCURACY
#ACCURACY = accuracy_score(y_test, y_pred)

In [20]:
def xgboost(max_depth, n_estimators, x_train, x_test, y_train, y_test):
    xgb = XGBClassifier(max_depth=max_depth, n_estimators=n_estimators)
    xgb.fit(x_train, y_train)
    y_pred = xgb.predict_proba(x_test)
    #return np.sqrt(mean_squared_error(y_test, y_pred))
    return roc_auc_score(y_test, y_pred[:,1])
#TAMBIEN PUEDEN USAR PARA VER EL ACCURACY
#ACCURACY = accuracy_score(y_test, y_pred)

In [49]:
def adaboost(nEstimators, learningRate, x_train, x_test, y_train, y_test):
    ada = AdaBoostClassifier(n_estimators=nEstimators, learning_rate=learningRate, random_state=0)
    ada.fit(x_train, y_train)
    y_pred = ada.predict_proba(x_test)
    return roc_auc_score(y_test, y_pred[:,1])
    #return np.sqrt(mean_squared_error(y_test, y_pred[:,1]))
#TAMBIEN PUEDEN USAR PARA VER EL ACCURACY
#ACCURACY = accuracy_score(y_test, y_pred)
#n_estimators is the number of models to iteratively train.
#learning_rate is the contribution of each model to the weights and defaults to 1. 
#Reducing the learning rate will mean the weights will be increased or decreased to a small degree, 
#forcing the model train slower (but sometimes resulting in better performance scores).

In [68]:
result = logisticReg(x_train, x_test, y_train, y_test)
result

0.5

In [69]:
result1 = knn(10, x_train, x_test, y_train, y_test)
result1

0.5

In [37]:
resultDecTree = decisionTree(150, x_train, x_test, y_train, y_test)
resultDecTree

0.560895563914086

In [39]:
resultXgboost = xgboost(10, 200,x_train, x_test, y_train, y_test )
resultXgboost

0.5117385200587032

In [10]:
score = logisticReg(x_train, x_test, y_train, y_test)
score

/home/martin/miniconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


0.3887532574552452

In [27]:
score1 = knn(10, x_train, x_test, y_train, y_test)
score1

0.5820849518841736

In [26]:
scoreDecTree = decisionTree(100, x_train, x_test, y_train, y_test)
scoreDecTree

0.5632381706853452

In [34]:
#MD:5 EST:100 0.8252031462716871
#MD:10 EST:100 0.7931231854354848
scoreXgboost = xgboost(10, 100, x_train, x_test, y_train, y_test )
scoreXgboost

0.7931231854354848

In [35]:
xgboost = XGBClassifier()

In [36]:
#Empieza el gridSearch
xgb_grid_param = {
    'max_depth': [3, 5, 7, 9], #[3,4,5,6,7,8,9], # 5 is good but takes too long in kaggle env
    'subsample': [0.6, 0.8], #[0.4,0.5,0.6,0.7,0.8,0.9,1.0],
    'colsample_bytree': [0.6, 0.8], #[0.5,0.6,0.7,0.8],
    'n_estimators': [100, 500, 1000], #[1000,2000,3000]
    'learning_rate': [0.1]
}

In [37]:
xgb_gs = GridSearchCV(estimator=xgboost, param_grid=xgb_grid_param, scoring="roc_auc", cv=5, n_jobs=1)

In [8]:
x_train2, x_test2, y_train2, y_test2 = train_test_split(data_array, label_array, test_size = 0.33, random_state = 1)

In [39]:
xgb_gs.fit(x_train2, y_train2)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid='warn', n_jobs=1,
       param_grid={'max_depth': [3, 5, 7, 9], 'subsample': [0.6, 0.8], 'colsample_bytree': [0.6, 0.8], 'n_estimators': [100, 500, 1000], 'learning_rate': [0.1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [40]:
#muestra el mejor hiperparametro
best_parameters = xgb_gs.best_params_
print(best_parameters)

{'colsample_bytree': 0.6, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.8}


In [41]:
xgb_gs.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.6, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.8)

In [42]:
#Muestra el mejor score
xgb_gs.best_score_

0.8349801057030514

In [50]:
scoreAda = adaboost(100, 0.1, x_train, x_test, y_train, y_test )
scoreAda

0.8277700847203846

In [54]:
def bagging(baseEstimator, nEstimators, learningRate, max_feature, bootstrap, bootstrap_feature, x_train, x_test, y_train, y_test):
    bag = BaggingClassifier(base_estimator=base_estimator, n_estimators=nEstimators, max_features=max_feature, bootstrap=bootstrap, bootstrap_features=bootstrap_feature, random_state=seed)
    bag.fit(x_train, y_train)
    y_pred = bag.predict_proba(x_test)
    #return np.sqrt(mean_squared_error(y_test, y_pred))
    return roc_auc_score(y_test, y_pred[:,1])
#TAMBIEN PUEDEN USAR PARA VER EL ACCURACY
#ACCURACY = accuracy_score(y_test, y_pred)

In [68]:
xgboostSuper = XGBClassifier(colsample_bytree=0.6, learning_rate=0.1, max_depth=3, n_estimators=100, subsample=0.8)

In [71]:
bag = BaggingClassifier(base_estimator=xgboostSuper, n_estimators=100, max_features=5, bootstrap=True, random_state=2)
bag.fit(x_train, y_train)
y_pred = bag.predict_proba(x_test)
scoreBag = roc_auc_score(y_test, y_pred[:,1])
scoreBag

0.7315288748054022

In [9]:
predict = XGBClassifier(colsample_bytree=0.6, learning_rate=0.1, max_depth=3, n_estimators=100, subsample=0.8)
predict.fit(x_train, y_train)
y_pred = predict.predict_proba(x_test)
scoreboost = roc_auc_score(y_test, y_pred[:,1])
scoreboost

0.8292636823739548

In [10]:
x_test

array([[ 2.,  2.,  0., ...,  1.,  0.,  0.],
       [ 4.,  4.,  3., ...,  8.,  0.,  0.],
       [ 2.,  0.,  1., ...,  3.,  0.,  0.],
       ...,
       [ 1.,  0.,  1., ...,  0.,  0.,  0.],
       [ 2.,  2.,  1., ...,  0.,  0.,  0.],
       [51.,  0.,  4., ..., 31.,  0.,  0.]])

In [11]:
kaggle_array

array([[372.,   0.,   2., ...,   0.,   0.,   0.],
       [  3.,   0.,   0., ...,   0.,   0.,   0.],
       [153.,   1.,   6., ...,  22.,   1.,   0.],
       ...,
       [  0.,   1.,   0., ...,   0.,   0.,   0.],
       [ 42.,  12.,   1., ...,   2.,   0.,   0.],
       [  4.,   1.,   1., ...,   0.,   0.,   0.]])

In [13]:
kaggle_predict = predict.predict_proba(kaggle_array)
kaggle_predict[:,1]

array([0.06132358, 0.08046189, 0.12535949, ..., 0.08243927, 0.06676186,
       0.00970416], dtype=float32)

In [18]:
kaggle_submit = {'person': kaggle["person"], 'label': kaggle_predict[:,1]}
kaggle_submitDF = pd.DataFrame(data=kaggle_submit)
kaggle_submitDF

,person,label
0,00091926,0.061324
1,00091a7a,0.080462
2,000ba417,0.125359
3,000e4d9e,0.033969
4,000e619d,0.043656
5,001001be,0.049080
6,0010e89a,0.006724
7,0016c4b5,0.004262
8,001804a2,0.008107
9,001a2273,0.007239


In [20]:
kaggle_submitDF.to_csv('setSubmit.csv', index=False)